# Example of DOV search methods for boringen

@pjhaest
date 20180503

## Use cases:
* Select boreholes in a bbox
* Select boreholes in a bbox with selected properties
* Select boreholes in a municipality
* Select boreholes in a municipality and return depth
* Get boreholes using info from wfs fields, not available in the standard output dataframe
* Get boreholes for a municipality where groundwater related data are available




In [1]:
%matplotlib inline
import os, sys
import inspect

In [2]:
# check pydov path
import pydov
print(pydov.__path__)

['c:\\_workingdir\\pydov\\pydov']


## Get information about code base

In [3]:
from pydov.search.boring import BoringSearch
boring = BoringSearch()

# information about the Boring type (In Dutch):
print(boring.get_description())

In de DOV-databank is elke waarneming van grondlagen een boring. Bij de meeste boringen wordt er met een boortoestel een gat gemaakt in de ondergrond om de verschillende grondlagen te kunnen beschrijven. Aan de hand van een boring krijg je een beeld van het materiaal in de ondergrond met toenemende diepte. Afhankelijk van het doel waarvoor de boring geplaatst wordt, zal men een geschikte boormethode toepassen. Boringen worden geplaatst voor verkennend bodemonderzoek, monstername van het sediment en/of grondwater, bepaling van bodemfysische parameters, milieuhygienisch onderzoek,... Afhankelijk van de diepte, soort materiaal, en het al dan niet boren tot onder de grondwatertafel kan men kiezen uit verscheidene systemen voor handmatig of machinaal te boren. Het bodemmateriaal dat vrijkomt, kan gebruikt worden om een profiel van de ondergrond op te stellen of om er grondmonsters van te nemen om verdere analyses op uit te voeren. Vaak is het de bedoeling een put uit te bouwen zodat water k

In [4]:
# information about the available fields for a Boring object
fields = boring.get_fields()
# print available fields
for f in fields.values():
    print(f['name'])

C:\ProgramData\Anaconda3\lib\site-packages\owslib\iso.py:117: FutureWarning: the .identification and .serviceidentification properties will merge into .identification being a list of properties.  This is currently implemented in .identificationinfo.  Please see https://github.com/geopython/OWSLib/issues/38 for more information
  FutureWarning)


generated_id
id
boornummer
pkey_boring
rapport
diepte_boring_tot
datum_aanvang
namen
putnummer
x
y
start_boring_mtaw
gemeente
uitvoerder
doel
methode
erkenning
opdrachtgever
informele_stratigrafie
formele_stratigrafie
lithologische_beschrijving
gecodeerde_lithologie
hydrogeologische_stratigrafie
quartaire_stratigrafie
geotechnische_codering
informele_hydrostratigrafie
doorheen_quartair
dikte_quartair
tertiair_onder_quartair
opdrachten
mv_mtaw
diepte_boring_van
boorgatmeting
diepte_methode_van
diepte_methode_tot
boormethode


In [5]:
# print information for a certain field
fields['diepte_boring_tot']

{'cost': 1,
 'definition': 'Maximumdiepte van de boring ten opzichte van het aanvangspeil, in meter.',
 'name': 'diepte_boring_tot',
 'notnull': False,
 'type': 'float'}

The cost is an arbitrary attribute to indicate if the information is retrieved from a wfs query (cost = 1), 
or from an xml (cost = 10)

In [6]:
# if an attribute can have several values, these are listed under 'values', e.g. for 'methode':
fields['methode']

{'cost': 1,
 'definition': "De methode waarmee de boring uitgevoerd werd. Heeft als waarde 'onbekend' indien de methode niet gekend is.",
 'name': 'methode',
 'notnull': True,
 'type': 'string',
 'values': ['avegaarboring',
  'droge boring',
  'edelmanboring',
  'geen boring',
  'gestoken boring',
  'graafmachine',
  'handboring',
  'kernboring',
  'lansen',
  'lepelboring',
  'luchthamer',
  'luchthevelboren of air-lift boren',
  'meerdere technieken',
  'omgek. spoelboring',
  'onbekend',
  'pulsboring',
  'ramguts',
  'ramkernboring',
  'rollerbit',
  'slagboring',
  'spade',
  'spiraalboring',
  'spoelboring',
  'steenboring',
  'trilboring',
  'voorput',
  'zuigboring']}

## Try-out of use cases

### Select boreholes in a bbox

In [7]:
# Get all borehole data in a bounding box (llx, llxy, ulx, uly)
# the pkey_boring link is not available below, but is in the df
df = boring.search(location=(153145, 206930, 153150, 206935))
print(df.head())

                                         pkey_boring     boornummer         x  \
0  https://www.dov.vlaanderen.be/data/boring/1974...  GEO-74/254-b1  153147.0   
1  https://www.dov.vlaanderen.be/data/boring/1974...  GEO-74/254-b1  153147.0   
2  https://www.dov.vlaanderen.be/data/boring/1974...  GEO-74/254-b1  153147.0   

          y  mv_mtaw  start_boring_mtaw   gemeente  diepte_boring_van  \
0  206931.0    14.12              14.12  Antwerpen                0.0   
1  206931.0    14.12              14.12  Antwerpen                0.0   
2  206931.0    14.12              14.12  Antwerpen                0.0   

   diepte_boring_tot datum_aanvang                          uitvoerder  \
0              14.05    1974-07-02  Rijksinstituut voor Grondmechanica   
1              14.05    1974-07-02  Rijksinstituut voor Grondmechanica   
2              14.05    1974-07-02  Rijksinstituut voor Grondmechanica   

   boorgatmeting  diepte_methode_van  diepte_methode_tot  boormethode  
0          Fa

The dataframe contains one borehole where two methodes ('boormethode') were applied for its construction. The available data are flattened to represent unique attributes per row of the dataframe

### Select boreholes in a bbox with selected properties

In [8]:
# list available query methods
methods = [i for i,j in inspect.getmembers(sys.modules['owslib.fes'], 
                                           inspect.isclass) 
           if 'Property' in i]
print(methods)
from owslib.fes import PropertyIsGreaterThanOrEqualTo

['PropertyIsBetween', 'PropertyIsEqualTo', 'PropertyIsGreaterThan', 'PropertyIsGreaterThanOrEqualTo', 'PropertyIsLessThan', 'PropertyIsLessThanOrEqualTo', 'PropertyIsLike', 'PropertyIsNotEqualTo', 'PropertyIsNull', 'SortProperty']


The property feature methodes listed above are available from the owslib module. These were not adapted for use in pydov.

In [9]:
# Get deep boreholes in a bounding box 
from owslib.fes import PropertyIsEqualTo
# the propertyname can be any of the fields of the boring object as shown on line 52 above, the literal is always a string, no 
# matter what its definition is in the boring object (string, float...)
query = PropertyIsGreaterThanOrEqualTo(
        propertyname='diepte_boring_tot', literal='2000')
df = boring.search(
    location=(200000, 211000, 205000, 214000),
    query=query
    )
print(df.head())

                                         pkey_boring  boornummer         x  \
0  https://www.dov.vlaanderen.be/data/boring/2016...  B/1-102785  201790.5   
1  https://www.dov.vlaanderen.be/data/boring/2016...  B/1-102786  201783.0   
2  https://www.dov.vlaanderen.be/data/boring/2016...  B/1-102786  201783.0   
3  https://www.dov.vlaanderen.be/data/boring/2016...  B/1-102786  201783.0   
4  https://www.dov.vlaanderen.be/data/boring/2016...  B/1-102786  201783.0   

          y  mv_mtaw  start_boring_mtaw gemeente  diepte_boring_van  \
0  212962.0     25.0               25.0      Mol                0.0   
1  212961.0     25.0               25.0      Mol                0.0   
2  212961.0     25.0               25.0      Mol                0.0   
3  212961.0     25.0               25.0      Mol                0.0   
4  212961.0     25.0               25.0      Mol                0.0   

   diepte_boring_tot datum_aanvang          uitvoerder  boorgatmeting  \
0             4341.0    2016-03

### Select boreholes in a municipality

In [10]:
query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Herstappe')
df = boring.search(query=query)
print(df.head())

                                         pkey_boring       boornummer  \
0  https://www.dov.vlaanderen.be/data/boring/2016...   kb33d106e-B236   
1  https://www.dov.vlaanderen.be/data/boring/1993...  kb41d120e-B1027   

          x         y  mv_mtaw  start_boring_mtaw   gemeente  \
0  224687.4  158191.0    124.0              124.0  Herstappe   
1  224820.0  157794.0    132.0              132.0  Herstappe   

   diepte_boring_van  diepte_boring_tot datum_aanvang  \
0                0.0                6.0          None   
1                0.0               50.0    1993-09-02   

                           uitvoerder  boorgatmeting  diepte_methode_van  \
0  Belgische Geologische Dienst (BGD)          False                 0.0   
1                      Peeters-Ramsel          False                 0.0   

   diepte_methode_tot      boormethode  
0                 6.0  gestoken boring  
1                50.0         onbekend  


### Select boreholes in a municipality and return depth

In [11]:
query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Gent')
df = boring.search(query=query,
                   return_fields=('diepte_boring_tot',))
print(df.head())
# get general info of dataframe
print(df.describe())
# depth of borehole equal to zero indicates missing values 
# a depth could be available in the interpretations

   diepte_boring_tot
0               42.0
1               32.0
2               59.0
3               11.0
4               27.0
       diepte_boring_tot
count        2630.000000
mean           15.525080
std            32.298611
min             0.000000
25%             0.000000
50%             6.095000
75%            20.000000
max           660.000000


### Get boreholes using info from wfs fields, not available in the standard output dataframe

To keep the dataframe size acceptable, not all wfs fields are included in the standard output. However, one can use this information to select boreholes as illustrated below.
The available wfs fields can be viewed in the object catalog of the Boringen wfs service, on-line [2018 05 04]: https://www.dov.vlaanderen.be/geonetwork/apps/tabsearch/index.html?hl=dut.
For example, make a selection of the boreholes in municipality the of Antwerp, for which a hydrogeological interpretation was performed:

In [33]:
from owslib.fes import And
query = And([PropertyIsEqualTo(propertyname='gemeente',
                               literal='Antwerpen'),
             PropertyIsEqualTo(propertyname='hydrogeologische_stratigrafie', 
                               literal='True')]
            )
df = boring.search(query=query)
print(df.head())

                                         pkey_boring    boornummer         x  \
0  https://www.dov.vlaanderen.be/data/boring/1937...   kb7d14e-B82  145457.0   
1  https://www.dov.vlaanderen.be/data/boring/1969...  kb7d14e-B110  143700.4   
2  https://www.dov.vlaanderen.be/data/boring/1966...  kb7d14e-B115  143067.0   
3  https://www.dov.vlaanderen.be/data/boring/1966...  kb7d14e-B117  144770.5   
4  https://www.dov.vlaanderen.be/data/boring/1970...  kb7d14e-B121  143597.8   

          y  mv_mtaw  start_boring_mtaw   gemeente  diepte_boring_van  \
0  224973.5      2.0                2.0  Antwerpen                0.0   
1  228086.3      2.0                2.0  Antwerpen                0.0   
2  227707.6      1.0                1.0  Antwerpen                0.0   
3  223430.6      6.0                6.0  Antwerpen                0.0   
4  223498.6     -1.9               -1.9  Antwerpen                0.0   

   diepte_boring_tot datum_aanvang     uitvoerder  boorgatmeting  \
0           

### Get boreholes for a municipality where groundwater related data are available

In [37]:
#  E.g. get the boreholes where 'gemeente' is 'Antwerp' and either 
# 'putnummer' is not empty 
# or 'doel' starts with 'Grondwater' 
# or 'erkenning' equals '2. Andere grondwaterwinningen'
from owslib.fes import PropertyIsLike
from owslib.fes import PropertyIsNull
from owslib.fes import Or
from owslib.fes import Not
query = And([PropertyIsEqualTo(propertyname='gemeente',
                               literal='Antwerpen'),
             Or([Not([PropertyIsNull(propertyname='putnummer')]),
                PropertyIsLike(propertyname='doel',
                               literal='Grondwater%'),
                PropertyIsEqualTo(propertyname='erkenning', 
                                  literal='2. Andere grondwaterwinningen')]
               )]
           )
df = boring.search(query=query)
print(df.head())


                                         pkey_boring       boornummer  \
0  https://www.dov.vlaanderen.be/data/boring/1997...    kb15d28e-B741   
1  https://www.dov.vlaanderen.be/data/boring/1996...    kb15d43w-B393   
2  https://www.dov.vlaanderen.be/data/boring/2016...       B/1-102776   
3  https://www.dov.vlaanderen.be/data/boring/2017...  1434-BG000106-9   
4  https://www.dov.vlaanderen.be/data/boring/2017...       B/1-102987   

          x         y  mv_mtaw  start_boring_mtaw   gemeente  \
0  154825.0  212445.0     5.00               5.00  Antwerpen   
1  151150.0  206550.0    18.09              18.09  Antwerpen   
2  153099.0  213577.0    -1.00              -1.00  Antwerpen   
3  149486.0  221697.0     5.98               5.98  Antwerpen   
4  149408.0  224256.0    -1.00              -1.00  Antwerpen   

   diepte_boring_van  diepte_boring_tot datum_aanvang  \
0                0.0               36.5    1997-08-21   
1                0.0              104.0    1996-06-26   
2    

## Visualize results

In [44]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Proj, transform

In [38]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    inProj = Proj(init='epsg:31370')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj, outProj, x1, y1)
    return x2, y2
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [47]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['boornummer'][loc]).add_to(marker_cluster)
fmap
